In [16]:
from math import log2
from sympy import isprime
from sympy import primitive_root, factorint
from sympy import factorint

def ceil(x):
    return int(x) if x == x // 1 else int(x // 1 + 1)

def find_group(n_bits_p, q):
    r_start_bits = ceil((n_bits_p - 1) - log2(q))
    r_start = 2**r_start_bits
    r_end = 2**n_bits_p - 1
    i = r_start
    while(i != r_end):
        if isprime(i * q + 1):
            print(f'i = {i}')
            break
        i += 1
    r = i
    p = q * r + 1
    assert(isprime(p))
    g = pow(2, r, p)
    assert(g != 1)
    assert(pow(g, q, p) == 1)
    return p, g, r

def primitive_nth_root_mod_q(q: int, n: int):
    assert (q-1) % n == 0, "n must divide p-1"
    g = primitive_root(q)             # primitive root modulo p
    root = pow(g, (q - 1) // n, q)        # candidate of order n
    # verify primitivity
    if pow(root, n, q) != 1:
        raise RuntimeError("candidate not an n-th root")
    for r in factorint(n).keys():
        if pow(root, n // r, q) == 1:
            raise RuntimeError("not primitive (has smaller order)")

    # get inverse
    inv_root = pow(root, q - 2, q)
    assert(pow(inv_root, n, q) == 1)
    return root, inv_root


In [17]:
def simple_group_54_1024():
    q = 18014398509506561
    n_bits_p = 1024
    p, g, r = find_group(n_bits_p, q)

    n = 2**12
    root_nth, root_nth_inv = primitive_nth_root_mod_q(q, n)
    root_2nth, root_2nth_inv = primitive_nth_root_mod_q(q, 2 * n)
    g1 = 45166903464448924280053243999855094667742421468529127038000390690990555413931211221819477288776566894331153249407833821645949924742511460070046075201900441040131011971807761465785928717328671867197258734612868374355511121959746617532947112651893101356660662278786690036661631961050681853982159875764050554114
    r1 = 4989600773829992505939281774675235167160739812342925804737541532336365734955082555252981441562650799135269463894280377252737218446399994046048621150492088786787399460885177398506151463611780180313959299437942577336618264065630845335441386156556402970639985534021305180832638133006171157561522
    p1 = 89884656743115800376065866080041538869586058840319453933641838155019455958415370128385800572962392718049474671151114996509417426331159167274173985578890984435612970736081155406155170101580895226642186535601576839882976877204283262818534938279529430261551494225065330535924818135470675393868825060316220145843
    assert(g == g1)
    assert(r == r1)
    assert(p == p1)
    assert(p == q * r + 1)

    assert(root_nth == 5194839201355896)
    assert(root_nth_inv == 3071585657230524)
    assert(root_2nth == 9455140237568613)
    assert(root_2nth_inv == 1909215453544473)

In [20]:
def bn254_and_simple_group_254_3072():
    """
    https://blog.lambdaclass.com/how-we-implemented-the-bn254-ate-pairing-in-lambdaworks/
    The BN254 (in Lambdaworks BN254Curve) is the Barreto-Naehrig pairing friendly elliptic curve E of the form
        y^2= x^3 + 3
    over a finite field Fp where:
        p = 36x^4 + 36x^3 + 24x^2 + 6x + 1 is the 254-bit prime number:

    p = 21888242871839275222246405745257275088696311157297823662689037894645226208583
    x = 4965661367192848881 (seed)

        t = 6x^2 + 1 is the trace of Frobenius.
        r = 36x^4 + 36x^3 + 18x^2 + 6x + 1 = p + 1 - t
    is the number of points in the curve E(Fp).
    """
    p0 = 21888242871839275222246405745257275088696311157297823662689037894645226208583
    x = 4965661367192848881
    t = 6*x**2 + 1
    q = p0 + 1 - t
    assert(q == 36*x**4 + 36*x**3 + 18*x**2 + 6*x + 1)
    n = 2**12
    assert(q % (2 * n) == 1)

    n_bits_p = 3072
    p, g, rr = find_group(n_bits_p, q)

    root_nth, root_nth_inv = primitive_nth_root_mod_q(q, n)
    root_2nth, root_2nth_inv = primitive_nth_root_mod_q(q, 2 * n)
    print(f'q = {q}')
    print(f'p = {p}')
    print(f'g = {g}')
    print(f'root_nth = {root_nth}')
    print(f'root_nth_inv = {root_nth_inv}')
    print(f'root_2nth = {root_2nth}')
    print(f'root_2nth_inv = {root_2nth_inv}')
    assert(p == q * rr + 1)

bn254_and_simple_group_254_3072()

i = 200690946458808773904043874740111401542900039936103016227009407591374163346911165241527841802563500750104510906972168308836037377191303333227164028316489208921028813129748071869276041856893613646664115564520707636712397361735495161798784566891551633519066371965923138046714355604778433220640049475589732292556099818141474224813246728993392529273181319285532965163342127111397222772164166268598066457751629407601046329894739817383696285892492136521096538891817191161821491557814957363207943636807603438786357458834207536003263061542562479689069772472080563159164338952011083149811240979224011378125188638666562397593015466678938221063495649948271155002147033845062155387212655959951620574197967822236876187888875650476119882442897357944176402014604149862767188887841563977776976026817411555346536601954854854193649299045126101616332094855131267336204
q = 21888242871839275222246405745257275088548364400416034343698204186575808495617
p = 439277217826969877924949742915389526100485496430484066091620

In [19]:
print(f'g = {g}')

NameError: name 'g' is not defined

In [15]:
from math import log2
r_start_bits = (3072 - 1) - log2(2**254 - 1)
r_start = 2**ceil(r_start_bits)
r_start


100345473229404386952021937370055700771450019968051508113504703795687081673455582620763920901281750375052255453486084154418018688595651666613582014158244604460514406564874035934638020928446806823332057782260353818356198680867747580899392283445775816759533185982961569023357177802389216610320024737794866146278049909070737112406623364496696264636590659642766482581671063555698611386082083134299033228875814703800523164947369908691848142946246068260548269445908595580910745778907478681603971818403801719393178729417103768001631530771281239844534886236040281579582169476005541574905620489612005689062594319333281198796507733339469110531747824974135577501073516922531077693606327979975810287098983911118438093944437825238059941221448678972088201007302074931383594443920781988888488013408705777673268300977427427096824649522563050808166047427565633667072